# Recognizing Persian Digits

In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
import random
import utils
from PIL import Image
from IPython.display import display
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
%matplotlib inline

In [2]:
root = './data/Digits/isolated digits/'

## Contents
* Loading and Preprocessing Data

## Loading and Preprocessing Data

First we will load and normalize data, then we resize all of the images in (28save it to binary .npy file :

In [3]:
print(utils.maybe_save(root,0))

Pickling ./data/Digits/isolated digits/0.npy.
Full dataset tensor: (975, 28, 28)
Mean: 0.371464
Standard deviation: 0.206846
Pickling ./data/Digits/isolated digits/1.npy.
Full dataset tensor: (942, 28, 28)
Mean: 0.357408
Standard deviation: 0.240735
Pickling ./data/Digits/isolated digits/2.npy.
Full dataset tensor: (990, 28, 28)
Mean: 0.373154
Standard deviation: 0.226939
Pickling ./data/Digits/isolated digits/3.npy.
Full dataset tensor: (945, 28, 28)
Mean: 0.38771
Standard deviation: 0.208011
Pickling ./data/Digits/isolated digits/4.npy.
Full dataset tensor: (958, 28, 28)
Mean: 0.40026
Standard deviation: 0.192274
Pickling ./data/Digits/isolated digits/5.npy.
Full dataset tensor: (994, 28, 28)
Mean: 0.373855
Standard deviation: 0.20505
Pickling ./data/Digits/isolated digits/6.npy.
Full dataset tensor: (962, 28, 28)
Mean: 0.40089
Standard deviation: 0.195148
Pickling ./data/Digits/isolated digits/7.npy.
Full dataset tensor: (956, 28, 28)
Mean: 0.389926
Standard deviation: 0.216767
Pick